In [ ]:
import MeCab
m = MeCab.Tagger('-d /usr/lib/mecab/dic/mecab-ipadic-neologd')

In [ ]:
import pandas as pd

csv_df = pd.read_csv('example.csv',encoding="utf8") 
len(csv_df)
csv_df.info()
csv_df.head()

In [ ]:
import jaconv
wakatis = []
for w in csv_df['歳出事業']:
    words1 = []
    w = jaconv.z2h(w, digit=True, ascii=True) # 半角、小文字に統一
    for c in m.parse(w).splitlines()[:-1]:  #事業名を分割し解析結果を配列にして繰り返す
        surface, feature = c.split('\t')    #分割された事業名と結果をそれぞれ格納 
        if '名詞' in feature:             #名詞ならそれをwords1に追加
            words1.append(surface)
            
    str1 = ' '.join(words1) #分割された事業名を空白で区切る
    wakatis.append(str1)    #区切られた事業名を配列に格納
# wakatis
csv_df['wakati'] = wakatis
learn_df = csv_df[['歳出事業コード','歳出事業','wakati']]
learn_df.info()
learn_df.head()

In [ ]:
def find_similarity(x, y):
#    l = []
#    for i in range(1, len(x)+1):
#        for j in range(0, len(x)+1-i):
#            l.append(x[j:j+i])
        
    d0 = []
    for n in y:
        w = []
        for word in x:
            if word in n:
                w.append(word)
        d0.append(w)
    
    d1 = []    
    for v in d0:
        a = 1
        point = 0
        for word2 in v:
            for list_w in d0:
                if word2 in list_w:
                    a += 1
        
            point += len(word2)/a
        
        d1.append(point)
        
    return d1

d = find_similarity(wakatis[1], wakatis)
csv_df['ポイント'] = d
learn_df = csv_df[['歳出事業コード','歳出事業','ポイント']]
learn_df.info()
learn_df.head()

In [ ]:
rank = [['', 0], ['', 0], ['', 0]]
for name, point in zip(csv_df['歳出事業'], csv_df['ポイント']):
    if name == csv_df['歳出事業'][1]:
        continue
    if rank[0][1] <= point:
        rank[2] = rank[1]
        rank[1] = rank[0]
        rank[0] = [name, point]
        
    else:
        if rank[1][1] <= point:
            rank[2] = rank[1]
            rank[1] = [name, point]
        else:
            if rank[2][1] <= point:
                rank[2] = [name, point]

print(rank)

In [ ]:
import numpy as np
def find_similarity(x, y):
#    l = []
#    for i in range(1, len(x)+1):
#        for j in range(0, len(x)+1-i):
#            l.append(x[j:j+i])
        
    d0 = []
    for n in y:
        w = []
        for word in x:
            if word in n:
                w.append(word)
        d0.append(w)
    
    d1 = []    
    for v in d0:
        a = 1
        point = 0
        for word2 in v:
            for list_w in d0:
                if word2 in list_w:
                    a += 1
        
            point += len(word2)/a
        
        d1.append(point)
        
    return d1

for i in range(len(wakatis)):
    d = find_similarity(wakatis[i], wakatis)
    d = np.array(d)

    topn_indices = np.argsort(d)[::-1][:10]
    print(topn_indices)
    for sim, tweet in zip(d[topn_indices], np.array(csv_df['歳出事業'])[topn_indices]):
        print("({:.2f}): {}".format(sim, "".join(tweet.split())))